/sps/km3net/users/cdesio/antares_seawiet/ARCA/trigger_optimisation(0)>cat E_correction_v2.cc 
double E_correction(double E_fit)
{
// **** Input ****
// E_fit  -> Reconstructed energy    // in GeV

// **** Output ****
// E_fit_corr  -> Reconstructed energy corrected    // in GeV

// R. Coniglione 9-3-2016
// Correct the reconstructed energy estimated by JEnergy
//
// Correction based on the fit of the median distributions of log10(E_mu) as a function of the reconstructed energy log10(E_fit). 
// see PhD thesis of Agata Trovato
// 
// median values at low energy estimated from the weighted spectra (nu_atm and E^-2 spectra)
// 100 files of nu_muCC used for the fit of the MC data 

      double logE_fit =log10(E_fit);
      double Fit_median=0.;

    
//      fit for E_fit from 10^4 to 10^7 GeV

/*
p0                        =     -8.72485   +/-   25.1183     
p1                        =      5.16495   +/-   25.1006     
p2                        =    -0.174627   +/-   9.93259     
p3                        =    -0.198979   +/-   1.94585     
p4                        =    0.0385912   +/-   0.188767    
p5                        =  -0.00213019   +/-   0.00725649  
*/
      double A0 = -8.72485;
      double A1 =  5.16495 ;          
      double A2 = -0.174627;         
      double A3 = -0.198979;           
      double A4 =  0.0385912 ;           
      double A5 = -0.00213019;           
     

//      fit for E_fit from 10^2.8 to 10^3.5

/*
p0                        =      1081.22   +/-   70308.3     
p1                        =     -1336.46   +/-   83940.7     
p2                        =      614.871   +/-   37521.8     
p3                        =     -124.755   +/-   7442.58     
p4                        =      9.43893   +/-   552.718     
*/
      double B0 =1081.22;
      double B1 =   -1336.46;          
      double B2 = 614.871 ;         
      double B3 =  -124.755;           
      double B4 =   9.43893;           
      

//      fit for E_fit from 10^3.5 to 10^4
/*
p0                        =     -60.3619   +/-   5169.48     
p1                        =      45.7012   +/-   4091.37     
p2                        =     -11.0884   +/-   1077.53     
p3                        =     0.923034   +/-   94.4355     
*/

      double C0 =-60.3619;
      double C1 = 45.7012;          
      double C2 =-11.0884 ;         
      double C3 =  0.923034;           

// linear extrapolation for 
//     E_fit <10^2.8 GeV
/*
p0                        =     -7.20045   +/-   23.0366     
p1                        =      3.03519   +/-   8.03504     
*/

      double D0 =-7.20045;
//      double D1 = 3.03519 ;    original      
      double D1 = 3.01 ;    // by hand to match the connection point

    // linear extrapolation for 
//      E_fit >10^7 GeV

//p0                        =     0.375618   +/-   32.7462     
//p1                        =      1.01931   +/-   4.8909      

      double E0 = 0.375618;
      double E1 = 1.01931;          

      if(logE_fit>1   && logE_fit<=2.8)Fit_median = D0+D1*pow(logE_fit,1);
      if(logE_fit>2.8 && logE_fit<=3.5)Fit_median = B0+B1*pow(logE_fit,1)+B2*pow(logE_fit,2)+B3*pow(logE_fit,3)+B4*pow(logE_fit,4);
      if(logE_fit>3.5 && logE_fit<=4)  Fit_median = C0+C1*pow(logE_fit,1)+C2*pow(logE_fit,2)+C3*pow(logE_fit,3);
      if(logE_fit>4   && logE_fit<7)   Fit_median = A0+A1*pow(logE_fit,1)+A2*pow(logE_fit,2)+A3*pow(logE_fit,3)+A4*pow(logE_fit,4)+A5*pow(logE_fit,5);
      if(logE_fit>=7)Fit_median = E0+E1*pow(logE_fit,1);
      if(logE_fit<=1)Fit_median = logE_fit; //correction not applied

      double E_fit_corr=pow(10,Fit_median);

      return E_fit_corr;
}


In [6]:
import ROOT
import root_numpy as rnp
import numpy as np
from dependencies import root_dependencies
root_dependencies()

In [7]:
rootfile = "../../ROOT_files/km3_v4_numucc_1.evt.JEnergy.root"

In [15]:
reco_en = np.hstack(rnp.root2array(rootfile, treename="EVT", branches="vector<JFIT::JFit>.__E"))

In [26]:
def E_corr(reco_energy):
    logE_fit = np.log10(reco_energy)

    #fit for E_fit from 10^4 to 10^7 GeV

    '''
    p0                        =     -8.72485   +/-   25.1183     
    p1                        =      5.16495   +/-   25.1006     
    p2                        =    -0.174627   +/-   9.93259     
    p3                        =    -0.198979   +/-   1.94585     
    p4                        =    0.0385912   +/-   0.188767    
    p5                        =  -0.00213019   +/-   0.00725649  
    '''
    A0 = -8.72485
    A1 =  5.16495          
    A2 = -0.174627
    A3 = -0.198979
    A4 =  0.0385912 
    A5 = -0.00213019

    #fit for E_fit from 10^2.8 to 10^3.5

    '''
    p0                        =      1081.22   +/-   70308.3     
    p1                        =     -1336.46   +/-   83940.7     
    p2                        =      614.871   +/-   37521.8     
    p3                        =     -124.755   +/-   7442.58     
    p4                        =      9.43893   +/-   552.718     
    '''
    B0 =1081.22
    B1 =   -1336.46
    B2 = 614.871
    B3 =  -124.755
    B4 =   9.43893
    B3 =  -124.755
    B4 =   9.43893
    #      fit for E_fit from 10^3.5 to 10^4
    '''
    p0                        =     -60.3619   +/-   5169.48     
    p1                        =      45.7012   +/-   4091.37     
    p2                        =     -11.0884   +/-   1077.53     
    p3                        =     0.923034   +/-   94.4355     
    '''

    C0 =-60.3619
    C1 = 45.7012
    C2 =-11.0884 
    C3 =  0.923034

    # linear extrapolation for E_fit <10^2.8 GeV
    '''
    p0                        =     -7.20045   +/-   23.0366     
    p1                        =      3.03519   +/-   8.03504     
    '''

    D0 =-7.20045
    #D1 = 3.03519    original      
    D1 = 3.01  #by hand to match the connection point

    #linear extrapolation for E_fit >10^7 GeV

    #p0                        =     0.375618   +/-   32.7462     
    #p1                        =      1.01931   +/-   4.8909      

    E0 = 0.375618
    E1 = 1.01931
    
    E_fit_corr = np.zeros(logE_fit.shape)
    for i, en in enumerate(logE_fit):
        if(en >1. and en <= 2.8):
            Fit_median = D0+D1*pow(en,1)
        elif(en >2.8 and en<=3.5):
            Fit_median = B0+B1*pow(en,1)+B2*pow(en,2)+B3*pow(en,3)+B4*pow(en,4)
        elif(en>3.5 and en<=4.):
            Fit_median = C0+C1*pow(en,1)+C2*pow(en,2)+C3*pow(en,3)
        elif(en>4. and en<7.):
            Fit_median = A0+A1*pow(en,1)+A2*pow(en,2)+A3*pow(en,3)+A4*pow(en,4)+A5*pow(en,5)
        elif(en>=7.):
            Fit_median = E0+E1*pow(en,1)
        elif(en<=1.):
            Fit_median = en # correction not applied

        E_fit_corr[i]=pow(10,Fit_median)

    return E_fit_corr

In [32]:
E_fit_corr = E_corr(reco_en)

In [28]:
mc_en = rnp.root2array(rootfile, treename="MONTECARLO", branches="MONTECARLO.neutrino_.E_")

In [37]:
mc_trklst_E = np.hstack(rnp.root2array(rootfile, treename="MONTECARLO", branches="MONTECARLO.TrackList().E_[0]"))

In [30]:
mc_en[:10]

array([ 121.79 ,  167.372,  141.919,  112.132,  141.27 ,  172.358,
        190.802,  188.589,  296.37 ,  197.317])

In [38]:
for i in range(10):
    print(reco_en[i], E_fit_corr[i], mc_en[i], mc_trklst_E[i])

(24.205295225621221, 0.00092282692181570804, 121.79000000000001, 82.195599999999999)
(131.50714764890017, 0.15051769813212942, 167.37200000000001, 124.831)
(57.859219497161142, 0.0127142702461934, 141.91900000000001, 71.061499999999995)
(3027.7708778014539, 2025.7018700671338, 112.13200000000001, 86.730400000000003)
(1888.0720009000031, 693.21616644556343, 141.27000000000001, 105.536)
(8.6680261233903817, 8.6680261233903817, 172.358, 163.08199999999999)
(74.401212276759722, 0.027102280502718599, 190.80199999999999, 64.480000000000004)
(51.854076163516645, 0.0091421118991249616, 188.589, 45.424700000000001)
(291.46656899577556, 1.6518177687819215, 296.37, 236.58000000000001)
(1742.847983418379, 542.95535390636212, 197.31700000000001, 161.82300000000001)


In [35]:
pow?